In [9]:
import getpass
import os

from dotenv import load_dotenv
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.runnables import RunnableSequence
from langchain_mistralai import ChatMistralAI
from langchain_openai import ChatOpenAI

In [10]:
os.environ.pop("LANGCHAIN_API_KEY", None) if "LANGCHAIN_API_KEY" in os.environ else None
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ.pop("MISTRAL_API_KEY", None) if "MISTRAL_API_KEY" in os.environ else None
os.environ.pop("OPENAI_API_KEY", None) if "OPENAI_API_KEY" in os.environ else None

# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your Langchain API key: ")
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter your Mistral API key: ")
# os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

load_dotenv()

True

## Using Language Models

In [17]:
model: ChatMistralAI = ChatMistralAI(
    model="mistral-small-latest", temperature=0, api_key=os.environ["MISTRAL_API_KEY"]
)
# model: ChatOpenAI = ChatOpenAI(
#     model="gpt-3.5-turbo", temperature=0, api_key=os.environ["OPENAI_API_KEY"]
# )

messages: list[HumanMessage | SystemMessage] = [
    SystemMessage(content="Translate the following from English into Italian."),
    HumanMessage(content="Hi!"),
]

response: AIMessage = model.invoke(messages)
print(response)

content='Ciao!' response_metadata={'token_usage': {'prompt_tokens': 16, 'total_tokens': 19, 'completion_tokens': 3}, 'model': 'mistral-small-latest', 'finish_reason': 'stop'} id='run-f9c18456-7586-4ae9-913b-c4ffb362f235-0'


## Output Parsers

In [12]:
parser: StrOutputParser = StrOutputParser()
print(parser.invoke(response))

Ciao!


In [13]:
chain: RunnableSequence = model | parser
chain.invoke(messages)

'Ciao!'

## Prompt Templates

In [14]:
system_template: str = "Translate the following into {language}."

prompt_template: ChatPromptTemplate = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)
prompt: ChatPromptValue = prompt_template.invoke({"language": "Italian", "text": "Hi!"})
print(prompt)

messages: list[HumanMessage | SystemMessage] = prompt.to_messages()
print(messages)

messages=[SystemMessage(content='Translate the following into Italian.'), HumanMessage(content='Hi!')]
[SystemMessage(content='Translate the following into Italian.'), HumanMessage(content='Hi!')]


## LCEL

In [16]:
chain = prompt_template | model | parser

response = chain.invoke({"language": "Italian", "text": "Hi!"})
print(response)

Ciao!
